In [6]:
from google.cloud import dataplex_v1
from google.oauth2 import service_account

client = dataplex_v1.DataplexServiceClient()

metadata_service_client = dataplex_v1.MetadataServiceClient()
data_scan_client = dataplex_v1.DataScanServiceClient()

# Proyecto
project_id = 'vanti-poc-440213'
location = 'us-central1'

parent = f"projects/{project_id}/locations/{location}"

In [8]:
# Listar los lagos
lakes = client.list_lakes(parent=parent)

print("Lagos en el Proyecto:")
for lake in lakes:
    print(f"\nLago: {lake.name}")
    
    # Listar las zonas en el lago
    zones = client.list_zones(parent=lake.name)
    for zone in zones:
        print(f"  Zona: {zone.name}")
        
        # Listar las entidades en la zona
        entities = metadata_service_client.list_entities(parent=zone.name)
        for entity in entities:
            print(f"    Entidad: {entity.name}")
            
            # Aquí podrías obtener más detalles de la entidad si es necesario
            
# Listar los DataScans de Calidad de Datos
data_scans = data_scan_client.list_data_scans(parent=parent)

print("\nDataScans de Calidad de Datos:")
for data_scan in data_scans:
    # Verificar si es un DataScan de calidad de datos
    if data_scan.data_quality_spec is not None:
        print(f"\nDataScan ID: {data_scan.name}")
        # Obtener detalles del DataScan
        data_scan_detail = data_scan_client.get_data_scan(name=data_scan.name)
        # Acceder a las reglas de calidad
        quality_spec = data_scan_detail.data_quality_spec
        for rule in quality_spec.rules:
            print(f"  - Regla: {rule.rule_type}")

Lagos en el Proyecto:

Lago: projects/vanti-poc-440213/locations/us-central1/lakes/lake-poc
  Zona: projects/vanti-poc-440213/locations/us-central1/lakes/lake-poc/zones/raw
    Entidad: projects/891624744651/locations/us-central1/lakes/lake-poc/zones/raw/entities/raw_customers
    Entidad: projects/891624744651/locations/us-central1/lakes/lake-poc/zones/raw/entities/raw_seguimientos

DataScans de Calidad de Datos:

DataScan ID: projects/vanti-poc-440213/locations/us-central1/dataScans/raw-customer-data-quality-scan

DataScan ID: projects/vanti-poc-440213/locations/us-central1/dataScans/raw-customers-profile-scan

DataScan ID: projects/vanti-poc-440213/locations/us-central1/dataScans/raw-profile-scan


JsonVanti = r'c:\Users\danielpiamba\Downloads\vanti-poc-440213-e5ddc44ddc21.json'


In [ ]:
from google.cloud import dataplex_v1
from google.protobuf import field_mask_pb2

# Configurar el cliente
data_scan_client = dataplex_v1.DataScanServiceClient()

# Especifica tu proyecto, ubicación y DataScan
project_id = 'vanti-poc-440213'
location = 'us-central1'
data_scan_id = 'raw_transacciones_data_quality_scan'
data_scan_name = f"projects/{project_id}/locations/{location}/dataScans/{data_scan_id}"

# Obtener el DataScan
data_scan = data_scan_client.get_data_scan(name=data_scan_name)

# Identificar la regla a modificar
index = None
for idx, rule in enumerate(data_scan.data_quality_spec.rules):
    if rule.rule_type == dataplex_v1.DataQualityRule.RuleType.NULL_EXPECTATION:
        # Suponiendo que queremos modificar esta regla
        index = idx
        break

if index is not None:
    # Modificar el parámetro deseado
    data_scan.data_quality_spec.rules[index].threshold = 0.95 

    # Especificar los campos que han cambiado
    update_mask = field_mask_pb2.FieldMask(paths=["data_quality_spec.rules"])

    # Crear el DataScan actualizado
    updated_data_scan = dataplex_v1.DataScan(
        name=data_scan.name,
        data_quality_spec=data_scan.data_quality_spec,
    )

    # Realizar la actualización
    operation = data_scan_client.update_data_scan(
        data_scan=updated_data_scan,
        update_mask=update_mask,
    )

    # Esperar a que la operación se complete
    result = operation.result()

    print("La regla de calidad de datos ha sido actualizada exitosamente.")

    # Verificar el cambio
    updated_data_scan = data_scan_client.get_data_scan(name=data_scan_name)
    updated_rule = updated_data_scan.data_quality_spec.rules[index]
    print(f"Nuevo Umbral: {updated_rule.threshold}")
else:
    print("No se encontró la regla especificada para modificar.")


In [10]:
from google.cloud import dataplex_v1
from google.protobuf import field_mask_pb2

# Configurar el cliente
data_scan_client = dataplex_v1.DataScanServiceClient()

# Especifica tu proyecto, ubicación y DataScan
project_id = 'vanti-poc-440213'
location = 'us-central1'
data_scan_id = 'raw-transacciones-data-quality-scan'
data_scan_name = f"projects/{project_id}/locations/{location}/dataScans/{data_scan_id}"

# Obtener el DataScan
data_scan = data_scan_client.get_data_scan(name=data_scan_name)

In [12]:
rules = data_scan.data_quality_spec.rules
rules

[]

In [13]:
print(data_scan)


name: "projects/vanti-poc-440213/locations/us-central1/dataScans/raw-transacciones-data-quality-scan"
uid: "e59c3e77-223e-4afb-a73a-e66b0e1b4695"
display_name: "raw_transacciones_data_quality_scan"
state: ACTIVE
create_time {
  seconds: 1730465540
  nanos: 189570271
}
update_time {
  seconds: 1730469242
  nanos: 812602745
}
data {
  entity: "projects/891624744651/locations/us-central1/lakes/lake-poc/zones/raw/entities/raw_transacciones"
}
execution_spec {
  trigger {
    on_demand {
    }
  }
}
execution_status {
}
type_: DATA_QUALITY



In [14]:
from google.cloud import dataplex_v1
from google.protobuf import field_mask_pb2

data_scan_client = dataplex_v1.DataScanServiceClient()

# Configurar el field mask para incluir data_quality_spec
field_mask = field_mask_pb2.FieldMask(paths=["data_quality_spec"])

# Obtener el DataScan con el field mask
data_scan = data_scan_client.get_data_scan(name=data_scan_name, read_mask=field_mask)

# Verificar las reglas de calidad
print(data_scan.data_quality_spec.rules)


TypeError: DataScanServiceClient.get_data_scan() got an unexpected keyword argument 'read_mask'

In [15]:
from google.cloud import dataplex_v1

data_scan_client = dataplex_v1.DataScanServiceClient()

# Obtener el DataScan completo
data_scan = data_scan_client.get_data_scan(name=data_scan_name)

# Verificar las reglas de calidad
print(data_scan.data_quality_spec.rules)


[]


In [7]:

from google.cloud import dataplex_v1

def listar_reglas_calidad(project_id, location, lake_id, data_scan_id):
  """Lista las reglas de calidad de un data scan job."""

  data_scan_client = dataplex_v1.DataScanServiceClient()
  name = f"projects/{project_id}/locations/{location}/lakes/{lake_id}/dataScans/{data_scan_id}/dataScanJobs/{data_scan_id}"  # Corrección aquí

  data_scan_job = data_scan_client.get_data_scan_job(name=name)

  # Accede a la especificación de calidad de datos desde el data scan job
  for rule in data_scan_job.data_quality_spec.rules:
      print(f"Regla: {rule.name}")
      print(f"  Descripción: {rule.description}")
      # ... imprime otras propiedades de la regla ...

# Ejemplo de uso
project_id = "vanti-poc-440213"
location = "us-central1"  # ej. "us-central1"
lake_id = "lake-poc"
data_scan_id = "raw-transacciones-data-quality-scan"

listar_reglas_calidad(project_id, location, lake_id, data_scan_id)

InvalidArgument: 400 Malformed collection name: 'lakes/dataScans/dataScanJobs'

In [9]:
from google.cloud import dataplex_v1

def listar_reglas_calidad(project_id, location, lake_id, data_scan_id, job_id):  # Añade job_id como argumento
  """Lista las reglas de calidad de un data scan job."""

  data_scan_client = dataplex_v1.DataScanServiceClient()
  name = f"projects/{project_id}/locations/{location}/lakes/{lake_id}/dataScans/{data_scan_id}/jobs/{job_id}"  # Corrección aquí

  data_scan_job = data_scan_client.get_data_scan_job(name=name)

  # Accede a la especificación de calidad de datos desde el data scan job
  for rule in data_scan_job.data_quality_spec.rules:
      print(f"Regla: {rule.name}")
      print(f"  Descripción: {rule.description}")
      # ... imprime otras propiedades de la regla ...

# Ejemplo de uso
project_id = "vanti-poc-440213"
location = "us-central1"  # ej. "us-central1"
lake_id = "lake-poc"
data_scan_id = "raw-transacciones-data-quality-scan"
job_id = "5bbec257-f912-436f-92ca-152448c5d51d"  # Añade el ID del job


listar_reglas_calidad(project_id, location, lake_id, data_scan_id, job_id)

InvalidArgument: 400 Malformed collection name: 'lakes/dataScans/jobs'

In [17]:
pip install --upgrade google-cloud-dataplex

Note: you may need to restart the kernel to use updated packages.


In [2]:
import google.cloud.dataplex

print(google.cloud.dataplex.__version__)

2.3.1
